# Imports

In [ ]:
## Logging and Shell
import logging
logging.basicConfig(
    level=logging.INFO,
    force=True
)

## Numeric imports
import pandas as pd

## File I/O
from pathlib import Path
import json

## Cheminformatics
from rdkit import Chem

# Param input

In [ ]:
mono_data_path : Path = Path('processed_monomer_data') / 'clean_smarts_digroup.csv'
rxn_mech_dir   : Path = Path('rxn_smarts')

rxns_from_smarts : bool = True#False

## Load processed monomer starting structures

In [ ]:
logging.info(f'Loading processed data from {mono_data_path.stem}')
digroup_table = pd.read_csv(mono_data_path, index_col=[0])
tables_by_chem = {
    chemistry : digroup_table[digroup_table['Chemistry'] == chemistry].dropna(axis=1).reset_index(drop=True)
        for chemistry in set(digroup_table['Chemistry'])
}

In [ ]:
digroup_table

## Load rxn mechanisms

In [ ]:
from polymerist.rdutils.reactions import reactions, reactors


reaction_pairs = {
    'NIPU' : ('cyclocarbonate', 'amine'),
    'urethane' : ('isocyanate', 'hydroxyl')
}

logging.info(f'Loading reaction mechanisms (from {"SMARTS" if rxns_from_smarts else "MDL files"})')
if rxns_from_smarts:
    with (rxn_mech_dir / 'rxn_smarts.json').open('r') as rxn_file:
        rxns = {
            chemistry : reactions.AnnotatedReaction.from_smarts(rxn_SMARTS)
                for chemistry, rxn_SMARTS in json.load(rxn_file).items()
        }
else:
    # from files
    rxns = {
        chemistry : reactions.AnnotatedReaction.from_rxnfile(rxn_mech_dir / f'{chemistry}.rxn')
            for chemistry in reaction_pairs.keys()
    }

# Polymerizing fragments

## Example for polymerization of individual molecule

In [ ]:
chemistry = 'urethane'
idx = 3

rxn_pathway = rxns[chemistry]
chem_tab = tables_by_chem[chemistry]
chem_entry = chem_tab.loc[idx]

reactants = {
    react_name : Chem.MolFromSmarts(chem_entry[react_name])
        for react_name in reaction_pairs[chemistry]
}

reactor = reactors.PolymerizationReactor(rxn_pathway)
for (dimer, pair) in reactor.propagate([i for i in reactants.values()]):
    display(dimer)
    for frag in pair:
        display(frag)
    print('='*50)

## Polymerize all SMILES into well-specified fragments with ports

In [ ]:
from polymerist.monomers import MonomerGroup
from polymerist.monomers import specification
from polymerist.rdutils.labeling import molwise


mono_info_dir  : Path = Path('monomer_files')
mono_info_dir.mkdir(exist_ok=True)

for chemistry, smarts_table in tables_by_chem.items():
    chem_dir = mono_info_dir / chemistry
    chem_dir.mkdir(exist_ok=True)

    for i, sample in smarts_table.iterrows():
        logging.info(f'Generating fragments for {chemistry} #{i}')
        # look up reactive groups and pathway by chemistry
        rxn_group_names = reaction_pairs[chemistry]
        rxn = rxns[chemistry]

        # read reactant monomers from digroup_table
        initial_reactants = []
        for j, group_name in enumerate(rxn_group_names):
            reactant = Chem.MolFromSmiles(sample[group_name], sanitize=False)
            for atom in reactant.GetAtoms():
                atom.SetProp('reactant_group', group_name)
            
            initial_reactants.append(reactant)
        monogrp = MonomerGroup()

        # first round of polymerization (initiation)
        reactor = reactors.PolymerizationReactor(rxn)
        for dimer, frags in reactor.propagate(initial_reactants):
            for assoc_group_name, rdfragment in zip(rxn_group_names, frags):
                # label and sanitize fragments
                molwise.clear_atom_isotopes(rdfragment, in_place=True)
                molwise.assign_ordered_atom_map_nums(rdfragment, in_place=True) # number monomers
                Chem.Kekulize(rdfragment, clearAromaticFlags=True) # convert aromatic bonds to single-double
                Chem.SanitizeMol(rdfragment, sanitizeOps=specification.SANITIZE_AS_KEKULE)

                # generate spec-compliant SMARTS
                raw_smarts = Chem.MolToSmarts(rdfragment)
                spec_smarts = specification.compliant_mol_SMARTS(smarts=raw_smarts)

                # record to monomer group
                affix = 'TERM' if MonomerGroup.is_terminal(rdfragment) else 'MID'
                monogrp.monomers[f'{assoc_group_name}_{affix}'] = [spec_smarts]

        monogrp.to_file(chem_dir / f'{chemistry}_{i}.json')